IAMSI -- 2021-2022
--------


# TME 09 : Règles d'association

<font color="RED" size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms**

- KRISNI Almehdi (3800519)
- ARICHANDRA Santhos (3802651)

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Contenu de l'archive téléchargée

Une fois détarée, l'archive **tme-09.tgz** crée le répertoire <code>tme09</code> contenant :
- ce notebook que vous lisez en ce moment et que vous allez compléter.
- un répertoire <code>dataset</code> contenant des bases d'exemples pour tester les algorithmes
- deux fichiers exécutables (Linux): <code>apriori32</code> et <code>fpgrowth32</code> (ces 2 programmes ont été développés par Christian Borgelt, plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance. Si vous n'êtes pas sous Linux, vous pouvez trouver une version pour votre OS ici:
    - <code>apriori</code> : https://borgelt.net/apriori.html
    - <code>fpgrowth</code> : https://borgelt.net/fpgrowth.html



#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce notebook complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un complément peut être envoyé **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyer le notebook tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

In [54]:
# Vérification de la version de Python utilisée:
import sys
sys.path # le path doit contenir python3.5 ou une version supérieure

['/Users/almehdikrisni/Desktop/UPMC/M1_DAC/S2/IAMSI/TME/TME9',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python39.zip',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9/lib-dynload',
 '',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9/site-packages',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9/site-packages/aeosa',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg',
 '/Users/almehdikrisni/opt/anaconda3/envs/KrisniEnv/lib/python3.9/site-packages/IPython/extensions',
 '/Users/almehdikrisni/.ipython',
 '/Users/almehdikrisni/.local/lib/python3.9/site-packages',
 '/Users/almehdikrisni/Desktop/UPMC/M1_DAC/S2/FootballGame-ML/gameEngine']

## Traitement d'une base d'apprentissage


### Chargement de la base

On commence par travailler sur la base exemple du fichier <code>exemple-1.txt</code> (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [55]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble Python plutôt que comme une liste. Il est ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [56]:
def chargeBase(filename) :
    # Création du dictionnaire
    d = dict()

    # On ouvre le fichier et on met à jour le dictionnaire
    with open(filename, 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=' ')
        i = 0
        for ligne in lecteur:
            i += 1
            d[i] = set(ligne)

    # On retourne le dictionnaire
    return d

In [57]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{1: {'a', 'b', 'c'},
 2: {'a', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'b', 'c', 'd'},
 5: {'b', 'c'},
 6: {'a', 'b', 'd'},
 7: {'d', 'e'},
 8: {'a', 'b', 'c', 'd'},
 9: {'c', 'd', 'e'},
 10: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [58]:
def noms_items(base) :
    # On crée un ensemble
    s = set()

    # On itère sur la base et on met à jour l'ensemble
    for i in base.items() :
        s = s.union(i[1])

    # On retourne l'ensemble
    return s

In [59]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

*Remarque*: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [60]:
def singletons(base) :
    # On crée une liste pour sauvegarder les singletons (itemsets de taille 1)
    l = []

    # On cherche les singletons
    for i in noms_items(base) :
        j = set([i])
        l.append(j)

    # On retourne la liste
    return l

In [61]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'c'}, {'e'}, {'a'}, {'d'}, {'b'}]

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [62]:
def comptage(base, itemset) :
    # On crée un compteur
    c = 0

    # On compte le nombre de transactions qui contiennent l'itemset
    for i in base.items() :
        if (itemset.issubset(i[1])) :
            c += 1

    # On retourne le compteur
    return c

In [63]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [64]:
def support(base, itemset) :
    # On retourne la valeur du support (nombr / nombre d'items dans la base)
    return comptage(base, itemset) / len(base)

In [65]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : " \
      +str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [71]:
def apriori_gen(itemlist) :
    # On crée la liste des itemsets candidats de longueur (k + 1)
    l = []

    # On itère sur la liste
    for i in range(len(itemlist)) :
        for j in itemlist[i:] :
            if itemlist[i] != j :
                tmp = itemlist[i].union(j)
                if (len(tmp) == len(j) + 1) and (tmp not in l) :
                    l.append(tmp)

    # On effectue une vérification sur les ensembles trouvés
    fl = l.copy() # Liste finale allant être retournée
    for cset in l :
        for j in cset :
            tmp = cset.copy()
            tmp.remove(j)
            if (tmp not in itemlist) and (cset in fl) :
                fl.pop(fl.index(cset))

    # On retourne la liste
    return fl

In [72]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

print("\nExemple: apriori_gen([{'a','b','d'}, {'b','c','d'}]) rend ")
print(apriori_gen([{'a','b','d'}, {'b','c','d'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'a', 'b'}, {'a', 'c'}, {'a', 'd'}, {'c', 'b'}, {'d', 'b'}, {'c', 'd'}]

Exemple: apriori_gen([{'a','b'},{'a','d'},{'b','d'},{'b','c'},{'c','d'}]) rend 
[{'a', 'd', 'b'}, {'c', 'd', 'b'}]

Exemple: apriori_gen([{'a','b','d'}, {'b','c','d'}]) rend 
[]


<font color="RED" size="+1">**[Q]**</font>  Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément et un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [76]:
def apriori(base, minsup) :
    # On récupère la liste originale des itemsets à étudier (donc les singletons)
    l = singletons(base)
    size = len(base)

    # Liste résultat allant contenir les tuples
    res = []

    # On itère sur la liste et on met à jour
    while l :
        # Création de liste pour la sauvegarde des itemset et des comptages
        itemsetList = []
        comptList = []

        # On sauvegarde les comptages dans l'ordre des itemset
        for itemset in l :
            itemsetList.append(itemset)
            comptList.append(comptage(base, itemset))

        # On met à jour les supports
        tmpItemset = [] # Liste allant servir à générer les nouveaux itemsets à étudier
        tmpSupport = [] # Liste allant servir à sauvegarder les nouvelles valeurs de support

        # On itère sur le dictionnaire
        for i in range(len(itemsetList)) :
            if (comptList[i] / size) >= minsup :
                tmpItemset.append(itemsetList[i])
                tmpSupport.append(comptList[i] / size)

        # On met à jour l
        l = apriori_gen(itemsetList)

        # On sauvegarde les résultats
        for i in range(len(tmpItemset)) :
            res.append((tmpItemset[i], tmpSupport[i]))

    # On retourne le résultat
    return res

In [77]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'c'}, 0.7), ({'e'}, 0.3), ({'a'}, 0.6), ({'d'}, 0.7), ({'b'}, 0.7), ({'c', 'a'}, 0.4), ({'c', 'd'}, 0.4), ({'c', 'b'}, 0.6), ({'e', 'd'}, 0.3), ({'a', 'd'}, 0.4), ({'a', 'b'}, 0.5), ({'d', 'b'}, 0.4), ({'c', 'a', 'b'}, 0.4), ({'c', 'd', 'b'}, 0.3), ({'a', 'd', 'b'}, 0.3)]


## Expérimentations

### Vérification avec la base du TD

Utiliser la fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que les itemsets obtenus sont corrects.

In [ ]:
# On reproduit la base étudiée pendant le TD

In [78]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
print(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 


NameError: name 'BaseTD' is not defined

### Comparaisons avec les implémentations apriori et fpgrowth

<font color="RED" size="+1">**[Q]**</font>  Tester votre fonction <code>apriori</code> sur les 3 bases données dans datasets et confronter les résultats obtenus avec ceux fournis par l'exécutable <code>apriori</code>.

In [ ]:
print("Résultat du chargement de 'mushrooms.txt', on obtient : ")
Base_mushrooms = chargeBase('datasets/mushrooms.txt',delimiteur=',')
#Base_mushrooms
print(len(Base_mushrooms),' transactions.')

Résultat du chargement de 'mushrooms.txt', on obtient : 
8416  transactions.


In [ ]:
print("Pour la BASE précédente :")
noms_items(Base_mushrooms)
print("Exemple: singletons(Base_mushrooms) rend :")
#singletons(Base_mushrooms)

Pour la BASE précédente :
Exemple: singletons(Base_mushrooms) rend :


In [ ]:
print(apriori(Base_mushrooms,0.9))

[({'WHITE'}, 0.9771863117870723), ({'PARTIAL'}, 1.0), ({'FREE'}, 0.9743346007604563), ({'ONE'}, 0.9230038022813688), ({'WHITE', 'PARTIAL'}, 0.9771863117870723), ({'WHITE', 'FREE'}, 0.9743346007604563), ({'WHITE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE'}, 0.9743346007604563), ({'PARTIAL', 'ONE'}, 0.9230038022813688), ({'FREE', 'ONE'}, 0.9001901140684411), ({'WHITE', 'PARTIAL', 'FREE'}, 0.9743346007604563), ({'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411), ({'WHITE', 'FREE', 'ONE'}, 0.9001901140684411), ({'PARTIAL', 'FREE', 'ONE'}, 0.9001901140684411), ({'FREE', 'WHITE', 'PARTIAL', 'ONE'}, 0.9001901140684411)]


In [ ]:
print("Résultat du chargement de 'titanic-red.csv', on obtient : ")
Base_titanic = chargeBase('datasets/titanic-red.csv',delimiteur=',')
print(len(Base_titanic),' transactions.')

Résultat du chargement de 'titanic-red.csv', on obtient : 
741  transactions.


In [ ]:
# décommenter les lignes suivantes:

#print("Pour la BASE Titanic :")
#noms_items(Base_titanic)

#print("Exemple: singletons(Base_titanic) rend :")
#singletons(Base_titanic)

In [ ]:
Lres = apriori(Base_titanic,0.2)

# tri sur les valeurs de supports:
#print(sorted(Lres, key=lambda data: data[1]))

# tri sur la taille des itemsets
#print(sorted(Lres, key=lambda data: len(data[0])))

for t in sorted(Lres, key=lambda data: data[1],reverse = True):
    if len(t[0])>1:
        print(t)


({'dead', 'male'}, 0.46963562753036436)
({'male', 'Southampton'}, 0.46288798920377866)
({'dead', 'Southampton'}, 0.41295546558704455)
({'dead', 'male', 'Southampton'}, 0.3684210526315789)
({'alive', 'female'}, 0.32388663967611336)
({'Southampton', '2nd'}, 0.30634278002699056)
({'alive', 'Southampton'}, 0.2982456140350877)
({'female', 'Southampton'}, 0.2483130904183536)
({'alive', '1st'}, 0.23076923076923078)
({'1st', 'male'}, 0.21997300944669365)
({'1st', 'Southampton'}, 0.20917678812415655)
({'male', '2nd'}, 0.20647773279352227)
({'alive', 'female', 'Southampton'}, 0.203778677462888)


<font color="RED" size="+1">**[Q]**</font>  Utiliser l'éxécutable <code>apriori</code> pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

<font color="RED" size="+1">**[Q]**</font>   Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


<font color="RED" size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

### Utilisation de fp-growth

<font color="RED" size="+1">**[Q]**</font>   Utiliser <code>fpgrowth</code> pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

<font color="RED" size="+1">**[Q]**</font>  En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des exécutables *apriori* et *fpgrowth*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori*, soit *fpgrowth*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
